In [3]:
#IMPORTANT LIB
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.chains.summarize import load_summarize_chain  
from transformers import T5Tokenizer, T5ForConditionalGeneration  
from transformers import pipeline
import base64 

In [ ]:
#FILE LOADER AND PREPROCESSING
model_name = "MBZUAI/LaMini-Flan-T5-248M"    
tokernzier = T5Tokenizer.from_pretrained(model_name)
base_model = T5ForConditionalGeneration.from_pretrained(model_name, device_map = 'auto', torch_dtype = torch.float32)   

def file_preprocessing(file):
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
    texts = text_splitter.split_documents(pages)
    final_text = ""
    for text in texts:
        print(text)
        final_text = final_text + text.page_content
    return final_text

SyntaxError: invalid syntax (2228056753.py, line 10)

In [ ]:
#LLM PIPELINE
def llm_pipeline(filepath):
    pipe_sum = pipeline(
        'summerization',
        model = base_model,
        tokernzier = tokernzier,
        max_length = 500,
        min_length = 50,
    )
    input_text = file_preprocessing(filepath)
    result = pipe_sum(input_text)
    result = result[0]['summary_text']
    return result

In [ ]:
#STREAMLIT CODE UI/UX
@st.cache_data
def display_PDF(file):
    with open(file, "rb") as f:
        base64_pdf = base64.b64decode(f.read().decode('utf-8'))

    pdf_display = F'<ifram src="data:application/pdf;base64,{base64_pdf}" width="100%" height="600" type="application/pdf"></iframe>'
    st.markdown(pdf_display, unsafe_allow_html=True)

st.set_page_config(layout='wide', page_title='Summerization App')

def main():
    st.title('Document Classification using LLM')
    uploaded_file = st.file_uploader("Upload your PDF", type=['pdf'])
    if uploaded_file is not None:
        if st.button("Summerization"):
            col1, col2 = st.columns(2)

            with col1:
                st.info("Uploaded PDF file")
            with col2:
                st.info("Summerization is below")    

In [ ]:
if __name__ == '__main__':
    main()